In [ ]:
# -*- coding:utf-8 -*-
# !/usr/bin/env python
 
# 这个文件用来测试是否能够训练

In [ ]:
import os
import random
import cv2
import tqdm
import torch
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog,DatasetCatalog

In [ ]:
# CHANGE HERE
project = "workpieces"
yaml_file = "/detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
weights = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_1x/137257794/model_final_b275ba.pkl"

register_coco_instances(project, {}, "./trainval.json", "./images")

In [ ]:
project_metadata = MetadataCatalog.get(project)
dataset_dicts = DatasetCatalog.get(project)

In [ ]:
torch.cuda.empty_cache()
print(project_metadata)

In [ ]:
# 从标签下显示图片
for d in random.sample(dataset_dicts, 3):
    pass

In [ ]:
cfg = get_cfg()
os.environ['HOME']
os.path.expandvars('$HOME')
home_path = os.path.expanduser('~')
cfg.merge_from_file(
    home_path+yaml_file
)
cfg.DATASETS.TRAIN = (project,)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = weights  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# 训练
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# The remain code is for test of predic.

In [ ]:
# predict
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = (project, )
predictor = DefaultPredictor(cfg)

In [ ]:
metadata = MetadataCatalog.get(
            cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused")
instance_mode = ColorMode.IMAGE
cpu_device = torch.device("cpu")
predictor = DefaultPredictor(cfg)
parallel = False

In [ ]:
def _frame_from_video(video):
    while video.isOpened():
        success, frame = video.read()
        if success:
            yield frame
        else:
            break


def run_on_video(video):
        """
        Visualizes predictions on frames of the input video.

        Args:
            video (cv2.VideoCapture): a :class:`VideoCapture` object, whose source can be
                either a webcam or a video file.

        Yields:
            ndarray: BGR visualizations of each video frame.
        """
        video_visualizer = VideoVisualizer(metadata, instance_mode)

        def process_predictions(frame, predictions):
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if "panoptic_seg" in predictions:
                panoptic_seg, segments_info = predictions["panoptic_seg"]
                vis_frame = video_visualizer.draw_panoptic_seg_predictions(
                    frame, panoptic_seg.to(cpu_device), segments_info
                )
            elif "instances" in predictions:
                predictions = predictions["instances"].to(cpu_device)
                vis_frame = video_visualizer.draw_instance_predictions(frame, predictions)
            elif "sem_seg" in predictions:
                vis_frame = video_visualizer.draw_sem_seg(
                    frame, predictions["sem_seg"].argmax(dim=0).to(cpu_device)
                )

            # Converts Matplotlib RGB format to OpenCV BGR format
            vis_frame = cv2.cvtColor(vis_frame.get_image(), cv2.COLOR_RGB2BGR)
            return vis_frame

        frame_gen = _frame_from_video(video)
        if parallel:
            buffer_size = predictor.default_buffer_size

            frame_data = deque()

            for cnt, frame in enumerate(frame_gen):
                frame_data.append(frame)
                predictor.put(frame)

                if cnt >= buffer_size:
                    frame = frame_data.popleft()
                    predictions = predictor.get()
                    yield process_predictions(frame, predictions)

            while len(frame_data):
                frame = frame_data.popleft()
                predictions = predictor.get()
                yield process_predictions(frame, predictions)
        else:
            for frame in frame_gen:
                yield process_predictions(frame, predictor(frame))

In [ ]:
# 从webcam中显示
cam = cv2.VideoCapture(2)
for vis in tqdm.tqdm(run_on_video(cam)):
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    cv2.imshow(project, vis)
    if cv2.waitKey(1) == 27:
        break  # esc to quit
cam.release()
cv2.destroyAllWindows()